In [ ]:
import os
import pickle
from pathlib import Path
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
from kooplearn.datasets import LogisticMap
logistic = LogisticMap(N = 20, rng_seed=0)

In [ ]:
#List and unpickle all files in "results/"
reports = []
for file in os.listdir("results/"):
    if file.endswith(".pkl"):
        with open("results/"+file, "rb") as f:
            reports.append(pickle.load(f))

def get_series(report, key):
    report = deepcopy(report)
    model_name = report.pop("name")
    series = np.zeros(len(report))
    for k, v in report.items():
        idx = int(k.split('_')[0]) - 2
        series[idx] = v.get(key, 0)
    feature_dims = np.arange(len(report)) + 2
    return feature_dims, series, model_name

def get_eigs(report):
    report = deepcopy(report)
    model_name = report.pop("name")
    eigs = []
    for k, v in report.items():
        idx = int(k.split('_')[0]) - 2
        eigs.append(v['eigenvalues'])
    feature_dims = np.arange(len(report)) + 2
    return feature_dims, eigs, model_name

In [ ]:
fig, ax = plt.subplots()
for report in reports:
    x, y, model_name = get_series(report, "hausdorff-dist")
    _, dy, _ = get_series(report, "hausdorff-dist_std")
    ax.plot(x, y, label=model_name)
    ax.fill_between(x, y - dy, y + dy, alpha=0.2)
ax.set_xlabel("Feature dimension")
ax.set_ylabel("Optimality gap")
ax.set_xmargin(0)
ax.legend(frameon=False)

In [ ]:
def plot_eigs(reports, feature_idx):
    fig, ax = plt.subplots(figsize=(5, 5))  
    #Activate grid
    ax.grid(True, which='both')
    #Plot unit circle
    theta = np.linspace(0, 2*np.pi, 100)
    ax.plot(np.cos(theta), np.sin(theta), color='k', lw=0.75)   
    for report in reports:
        x, eigs, model_name = get_eigs(report)
        eigs = eigs[feature_idx]
        print(f"{model_name} eigenvalues: {eigs}")
        ax.scatter(eigs.real, eigs.imag, label = model_name)
    ax.scatter(logistic.eig().real, logistic.eig().imag, marker='x', color='k', label = 'True eigenvalues')
    ax.set_xlabel('Real part')
    ax.set_ylabel('Imaginary part')
    ax.legend(frameon=False, loc='lower center', bbox_to_anchor=(0, 1, 1, 0), ncol=3, fontsize=8)
    ax.set_aspect('equal')

In [ ]:
plot_eigs(reports, 3)

In [ ]:
from pseudopy import NonnormalAuto
import scipy.integrate
N = 20

def koopman_el(i: int, j: int):
    def pairing(x):
        return logistic.noise_feature(x, i) * logistic.noise_feature_composed_map(x, j)
    return scipy.integrate.quad(pairing, 0, 1)[0]

K = np.array([[koopman_el(i, j) for j in range(N + 1)] for i in range(N + 1)])

In [ ]:
pseudo = NonnormalAuto(K, 1e-5, 1)

In [ ]:
pseudo

In [ ]:
pseudo.plot([10**k for k in range(-4, 0)], spectrum=logistic.eig())